# MM/ML protein ligand MD with OpenMM

This example runs molecular dynamics of a small molecule using AceFF as a ML potential. It uses [OpenMM](https://openmm.org/) and [OpenMM-ML](https://github.com/openmm/openmm-ml).

**You will need to change to an older Colab runtime environment**
For compatibility reasons you will need to use an older Colab runtime by clicking `runtime`→`change runtime type` and selecting `2025.07` from the `Runtime version` dropdown menu

**You should run this example on a GPU**
You can change to a GPU instance on Colab by clicking `runtime`→`change runtime type` and selecting `T4 GPU` from the `Hardware accelerator` dropdown menu.

In [ ]:
# Execute this cell to setup the python env in the Colab environment
if 'google.colab' in str(get_ipython()):
    print('Running on colab')
    # we need to remove the colab version of torch so we can use the conda-forge version
    !pip uninstall -y torch torchvision torchaudio
    !pip install -q condacolab
    import condacolab
    condacolab.install_mambaforge()
    !rm -rf /usr/local/conda-meta/pinned # remove pins so we can use cuda 11.8
    import os
    os.environ["CONDA_OVERRIDE_CUDA"] = "11.8"
    !mamba install openmm-torch=*=cuda118* # this version seems to be the most compatible
    !mamba install -c conda-forge openmmforcefields # this only has a conda package
else:
    print('Not running on colab.')
    print('Make sure you create and activate a new conda environment!')
    print('Please install openmm-torch and openmmforcefields, you will probably need to use conda for these.')

**Notes:**

- During this step on Colab the kernel will be restarted. This will produce the error message: "Your session crashed for an unknown reason. " This is normal and can be safely ignored. 
- Installing the necessary packages may take several minutes.

In [ ]:
if 'google.colab' in str(get_ipython()):
    # install the version of OpenMM-ML that has AceFF2.0
    !pip install git+https://github.com/openmm/openmm-ml.git@refs/pull/111/head --no-deps
    !pip install git+https://github.com/torchmd/torchmd-net.git
else:
    print("please install the versions of openmm-ml and torchmdnet above, you can use pip for these.")

In [ ]:
# download the example ligand and protein file
!wget https://raw.githubusercontent.com/Acellera/aceff_examples/refs/heads/main/notebooks/ejm_31_ligand.sdf
!wget https://raw.githubusercontent.com/Acellera/aceff_examples/refs/heads/main/notebooks/Tyk2.pdb

In [ ]:
import openmm.app as app
import openmm as mm
import openmm.unit as unit
from sys import stdout
from openmmforcefields.generators import GAFFTemplateGenerator
from openff.toolkit import Molecule
from openff.toolkit import Topology as offTopology
from openff.units.openmm import to_openmm as offquantity_to_openmm
from openmmml import MLPotential

# user supplied paths
ligand_sdf_path = "ejm_31_ligand.sdf"
protein_pdb_path = "Tyk2.pdb"

# MD settings
timestep = 1.0 * unit.femtosecond
hmr = 4 * unit.amu
total_steps = 10000
output_freq = 1000

# Load in the protein from a PDB file
protein_pdb = app.PDBFile(protein_pdb_path)

# load the ligand with OpenFF
ligand = Molecule.from_file(ligand_sdf_path)

# setup GAFF for the ligand
gaff = GAFFTemplateGenerator(molecules=ligand)

# Create an OpenMM ForceField object with AMBER ff14SB and TIP3P
ff = app.ForceField("amber/protein.ff14SB.xml", "amber14/tip3p.xml")
ff.registerTemplateGenerator(gaff.generator)

# make an OpenMM Modeller object with the protein
modeller = app.Modeller(protein_pdb.topology, protein_pdb.positions)

# make an OpenFF Topology of the ligand
ligand_off_topology = offTopology.from_molecules(molecules=[ligand])

# get the total ligand charge
ligand_charge = int(
    sum([atom.formal_charge.magnitude for atom in ligand_off_topology.atoms])
)

# convert it to an OpenMM Topology
ligand_omm_topology = ligand_off_topology.to_openmm()

# get the positions of the ligand
ligand_positions = offquantity_to_openmm(ligand.conformers[0])

# add the ligand to the Modeller
modeller.add(ligand_omm_topology, ligand_positions)

# solvate
modeller.addSolvent(ff, padding=1.0 * unit.nanometer, ionicStrength=0.15 * unit.molar)

# create OpenMM system
mm_system = ff.createSystem(
    modeller.topology,
    nonbondedMethod=app.PME,
    constraints=app.HBonds,
    nonbondedCutoff=1.0 * unit.nanometer,
    hydrogenMass=hmr,
    removeCMMotion=False,
)


# setup the ML Potential
# get the indicies of the ligand
chains = list(modeller.topology.chains())
ml_atoms = [atom.index for atom in chains[1].atoms()]

# create the ML potential with AceFF
potential = MLPotential(
    "aceff-2.0"
)

# create the MM/ML system
ml_system = potential.createMixedSystem(modeller.topology, mm_system, ml_atoms)
integrator = mm.LangevinMiddleIntegrator(
    300 * unit.kelvin, 1 / unit.picosecond, timestep
)
simulation = app.Simulation(modeller.topology, ml_system, integrator)


# set the positions
simulation.context.setPositions(modeller.positions)

# Save the toplogy as a PDB file.
with open(f"topology.pdb", "w") as output:
    app.PDBFile.writeFile(
        simulation.topology,
        simulation.context.getState(getPositions=True).getPositions(),
        output,
    )


print("Minimizing energy...")
simulation.minimizeEnergy(maxIterations=1000)

simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.reporters.append(app.XTCReporter(f"traj.xtc", output_freq))

simulation.reporters.append(
    app.StateDataReporter(
        stdout,
        output_freq,
        step=True,
        potentialEnergy=True,
        temperature=True,
        volume=True,
        speed=True,
    )
)

print("Running simulation...")
simulation.step(total_steps)